In [1]:
#CAPSTONE PROJECT; Debt Strategy

In [2]:
import pandas as pd 
import numpy as np
#from faker import Faker
import random 
import matplotlib.pyplot as plt 
import seaborn as sns

In [3]:
!pip install faker

In [4]:
btc_data = pd.read_csv(r"C:\Users\ebuca\OneDrive\Documents\DA13\Python\projects\Capstone_project\btc-usd-max.csv")
#btc_data['bitcoin_date'] = pd.to_datetime(btc_data['bitcoin_date']).dt.date 
print(btc_data.head(20))

   bitcoin_date    price  bitcoin_sats  one_dollar_sats
0      8/2/2017  2697.85     100000000            37067
1      8/3/2017  2777.37     100000000            36005
2      8/4/2017  2845.90     100000000            35138
3      8/5/2017  3237.07     100000000            30892
4      8/6/2017  3189.73     100000000            31351
5      8/7/2017  3356.71     100000000            29791
6      8/8/2017  3410.45     100000000            29322
7      8/9/2017  3330.90     100000000            30022
8     8/10/2017  3367.91     100000000            29692
9     8/11/2017  3562.59     100000000            28069
10    8/12/2017  3800.14     100000000            26315
11    8/13/2017  4023.25     100000000            24855
12    8/14/2017  4175.70     100000000            23948
13    8/15/2017  4149.31     100000000            24100
14    8/16/2017  4262.56     100000000            23460
15    8/17/2017  4286.92     100000000            23327
16    8/18/2017  4097.69     100000000          

In [5]:
data = pd.read_csv(r"C:\Users\ebuca\OneDrive\Documents\DA13\Python\projects\Capstone_project\updated_loan_data.csv")
#fake = Faker()
#def assign_random_names(df):
 #   df['Names'] = [fake.name() for _ in range(len(df))]
  #  cols = ['Names'] + [col for col in df.columns if col != 'Names']  # Reorder columns
   # return df[cols]
    
data.columns = data.columns.str.strip().str.lower()  # Remove spaces & lowercase
print(data.columns)  

# Converting 'issue_month' to a full date format, assuming the 1st of each month
data['issue_date'] = pd.to_datetime(data['issue_date'])


print(data['issue_date'].head())  # Check the first few values in 'issue_date'

Index(['names', 'profession', 'job_length', 'state', 'homeownership',
       'annual_income', 'monthly_wage', 'debt_to_income', 'total_credit_limit',
       'total_credit_utilized', 'loan_purpose', 'loan_amount', 'term',
       'interest_rate', 'installment', 'total_payment', 'ten_of monthly',
       'spent_strategy', 'issue_date', 'payoff_date'],
      dtype='object')
0   2018-02-01
1   2018-01-01
2   2018-01-01
3   2018-03-01
4   2018-02-01
Name: issue_date, dtype: datetime64[ns]


In [6]:





##this has a fuller table and will be exported out soon

import pandas as pd

# Convert dates to datetime format
btc_data['bitcoin_date'] = pd.to_datetime(btc_data['bitcoin_date'])
data['issue_date'] = pd.to_datetime(data['issue_date'])

# Remove rows with null 'issue_date' values in the 'data' dataframe before merging
data = data.dropna(subset=['issue_date'])  # Removes rows where 'issue_date' is null

# Convert 'term' column to integer to avoid float issue in range
data['term'] = data['term'].astype(int)

# Merge the two datasets on the issue_date and bitcoin_date (nearest match for each person)
merged_data = pd.merge_asof(data.sort_values('issue_date'), 
                            btc_data.sort_values('bitcoin_date'),
                            left_on='issue_date', right_on='bitcoin_date', 
                            direction='nearest')

# Initialize lists to store total sats and total dollars spent for each person
total_satoshis_per_customer = []
total_dollars_spent_per_customer = []

# Iterate over each row to calculate sats and dollars spent for each customer
for index, row in merged_data.iterrows():
    # Extract relevant data for the current customer
    monthly_spent = row['ten_of monthly']  # Monthly dollar spent
    issue_date = row['issue_date']
    term = row['term']
    
    # Initialize total sats counter and total dollars spent
    total_satoshis = 0
    total_dollars_spent = monthly_spent * term  # Correctly calculate total dollars spent over the loan term
    
    # Adjust the issue date to start from the next month
    start_date = issue_date + pd.DateOffset(months=1)
    
    for month_offset in range(term):
        # Calculate the first day of the month for this month in the term, starting from the next month
        month_date = start_date + pd.DateOffset(months=month_offset)
        month_date_first_day = month_date.replace(day=1)  # Ensure it's the 1st of the month
        
        # Find the corresponding 'one_dollar_sats' for this specific month
        sats_per_dollar = btc_data.loc[btc_data['bitcoin_date'] == month_date_first_day, 'one_dollar_sats'].values
        if sats_per_dollar.size > 0:
            sats_per_dollar = sats_per_dollar[0]
        else:
            sats_per_dollar = 0  # If no data for that specific day, assume 0 sats
        
        # Calculate satoshis bought for the current month
        satoshis_bought = monthly_spent * sats_per_dollar
        total_satoshis += satoshis_bought  # Add to the total satoshis bought

    # Append the total sats and total dollars spent for this customer
    total_satoshis_per_customer.append(total_satoshis)
    total_dollars_spent_per_customer.append(total_dollars_spent)

# Add the total satoshis and total dollars spent columns to the merged_data DataFrame
merged_data['total_satoshis'] = total_satoshis_per_customer
merged_data['total_dollars_spent'] = total_dollars_spent_per_customer

# Show the names, total sats accumulated, and total dollars spent
final_result = merged_data[['names', 'profession', 'loan_purpose', 'total_credit_utilized', 'loan_amount', 'interest_rate', 'term', 
                            'ten_of monthly', 'total_satoshis', 'total_dollars_spent', 'payoff_date']]

# Print out the result to check
print(final_result.head())






             names                  profession        loan_purpose  \
0      Ebuka Ajagu               Meme Director               other   
1   Sandra Sanders                          rn  debt_consolidation   
2  Danielle Bishop                 parts sales  debt_consolidation   
3     Jill Jackson  human resources supervisor  debt_consolidation   
4     Julie Fields        production scheduler  debt_consolidation   

  total_credit_utilized loan_amount  interest_rate  term  ten_of monthly  \
0                 10000      20,000           7.50    36            58.0   
1                14,848       25000          14.08    60            58.0   
2                23,143       21500          11.99    60            48.0   
3                80,746       23200           7.35    36            72.0   
4                45,651       15000          14.08    60            35.0   

   total_satoshis  total_dollars_spent payoff_date  
0      29950504.0               2088.0    9/1/2020  
1      32657074.

In [7]:




import pandas as pd

# Load the S&P 500 data
sp_data = pd.read_csv(r"C:\Users\ebuca\OneDrive\Documents\DA13\Python\projects\Capstone_project\sp_data.csv")

# Load the updated loan data
data = pd.read_csv(r"C:\Users\ebuca\OneDrive\Documents\DA13\Python\projects\Capstone_project\updated_loan_data.csv")

# Convert date columns to datetime
sp_data['s&p_date'] = pd.to_datetime(sp_data['s&p_date'])
data['issue_date'] = pd.to_datetime(data['issue_date'])

# Check and drop rows with null values in the issue_date column
if data['issue_date'].isnull().any():
    print("Warning: Null values found in 'issue_date' column. These rows will be removed.")
    data = data.dropna(subset=['issue_date'])

# Ensure both tables are sorted by their respective date columns before merging
sp_data = sp_data.sort_values('s&p_date')
data = data.sort_values('issue_date')

# Merge the two datasets on the issue_date and s&p_date (nearest match for each person)
merged_data = pd.merge_asof(
    data.sort_values('issue_date'), 
    sp_data.sort_values('s&p_date'),
    left_on='issue_date', right_on='s&p_date', 
    direction='nearest'
)

# Now you can proceed with further processing...
print(merged_data.head())


              Names         profession  job_length state homeownership  \
0       Ebuka Ajagu      Meme Director         5.0    TN          RENT   
1  Stacey Craig DDS    delivery driver        10.0    CO          RENT   
2    Christina King  operation manager        10.0    GA           OWN   
3      Philip Gates     clinical nurse        10.0    IL           OWN   
4        Jose Reyes        bar manager         6.0    NY          RENT   

  annual_income monthly_wage  debt_to_income  total_credit_limit  \
0        50,000        4,167            0.05             20000.0   
1        40,000        3,333           10.17             15262.0   
2       105,000        8,750           10.03            112755.0   
3       120,000       10,000           10.04             73599.0   
4        65,000        5,417           10.08             49200.0   

  total_credit_utilized  ... issue_date payoff_date   s&p_date     price  \
0                 10000  ... 2017-09-01    9/1/2020 2017-09-01  2,476.

In [8]:
# merging the table
